KINGS COUNTY HOUSE SALES ANALYSIS.

Please fill out:
* Student names:
1. Benson Kamau 
2. Kelvin Muchori
3. Nancy Chelagat
4. Breden Mugambi
5. Sally Kinyanjui

* Student pace: Full time

* Scheduled project review date/time: N/A

* Instructor name: Nikita Njoroge


BUSINESS UNDERSTANDING

This study aims to explore the relationship between key factors such as renovations, bedrooms, and other property attributes with housing prices in King County. By leveraging on the dataset available, we seek to uncover the specific impact of renovations on property valuations, considering variables such as the extent of renovations.

Additionally, we aim to investigate how the number of bedrooms, bathrooms, along with other structural features like square footage and lot size, view  influences housing prices.
This can help homeowners to add functionality and beauty to their property while simultaneously boosting its resale value.

Main Objective

To build and evaluate models using various combinations of the available features in the King County.


Specific Objectives

1.	What features of a home has a bigger impact its value in King County?
2.	Which features impacts a home’s value the most in King County?
3.	To evaluate which combinations of the available features in the dataset are the most impactful features for predicting sale price.


DATA UNDERSTANDING

This project analyzes data about homes sold in King County, Washington between May 2014 and May 2015 in order to make recommendations to relevant stakeholders.

This dataset is housed in the kc_house_data.csv file within the project's data folder and the columns outlined in the accompanying column_names.md file.


In [6]:
#import the relevant library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

# Define the DataFrame in the global scope
df = pd.DataFrame()

#create a function that loads data and gets the info about the data.
def load_and_get_info(file_path):
    """
    Load data from a CSV file and get information about the DataFrame.
    
    Parameters:
    - file_path (str): Path to the CSV file.
    
    Returns:
    - df_info (str): Information about the DataFrame.
    """
    # Load data
    global df
    df = pd.read_csv(file_path)
    
    # Get information about the DataFrame
    df_info = df.info()
    
    return df_info

file_path = 'data/kc_house_data.csv'  
data_info = load_and_get_info(file_path)
print(data_info)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

This dataset contains records of 21,597 home sales. It includes:

How much the house sold for

Square footage of the house's living space

Square footage of the lot

Number of floors, bedrooms, and bathrooms

The quality of the view from the house 

Whether the house is on a waterfront

The house's grade (relates to construction materials and quality)

The house's condition (relates to age and maintenance)

The year the house was built

The year the house was renovated

The location of the house



In [7]:
def check_dtypes(df):
    """
    Check data types and identify the kind of variable for each column in the DataFrame.
    
    Parameters:
    - df (DataFrame): Input DataFrame.
    
    Returns:
    - dtypes_info (DataFrame): DataFrame containing information about data types and variable types.
    """
    # Get data types of each column
    dtypes = df.dtypes
    
    # Identify variable types (numeric or categorical) for each column
    variable_types = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            variable_types.append('Numeric')
        else:
            variable_types.append('Categorical')
    
    # Create DataFrame to store information
    dtypes_info = pd.DataFrame({'Column': dtypes.index, 'Data Type': dtypes.values, 'Variable Type': variable_types})
    
    return dtypes_info

data_types_info = check_dtypes(df)
print(data_types_info)

           Column Data Type Variable Type
0              id     int64       Numeric
1            date    object   Categorical
2           price   float64       Numeric
3        bedrooms     int64       Numeric
4       bathrooms   float64       Numeric
5     sqft_living     int64       Numeric
6        sqft_lot     int64       Numeric
7          floors   float64       Numeric
8      waterfront    object   Categorical
9            view    object   Categorical
10      condition    object   Categorical
11          grade    object   Categorical
12     sqft_above     int64       Numeric
13  sqft_basement    object   Categorical
14       yr_built     int64       Numeric
15   yr_renovated   float64       Numeric
16        zipcode     int64       Numeric
17            lat   float64       Numeric
18           long   float64       Numeric
19  sqft_living15     int64       Numeric
20     sqft_lot15     int64       Numeric


## Data Preparation

In [8]:
# Define a custom function to transform the yr_renovated column and drop the column after transformation
def transform_and_drop_yr_renovated(df):
    """
    Transform yr_renovated column to read 'Yes' if there is a year recorded,
    and 'No' if there is no year in the record. Then drop the original
    yr_renovated column.
    
    Parameters:
    - df (DataFrame): Input DataFrame.
    
    Returns:
    - df_transformed (DataFrame): Transformed DataFrame with dropped column.
    """
    # Define a custom function to transform the yr_renovated column
    def transform_yr_renovated(value):
        if pd.notnull(value) and value != 0:  # Check if value is not NaN and not equal to 0
            return 'Yes'
        else:
            return 'No'

    # Apply the custom function to transform the yr_renovated column
    df['house_renovation'] = df['yr_renovated'].apply(transform_yr_renovated)
    
    # Drop the original yr_renovated column
    df = df.drop(columns=['yr_renovated'])
    
    return df

# Apply the custom function to transform and drop yr_renovated column
df = transform_and_drop_yr_renovated(df)

# Print the transformed DataFrame
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15,house_renovation
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,98178,47.5112,-122.257,1340,5650,No
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,98125,47.7210,-122.319,1690,7639,Yes
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,98028,47.7379,-122.233,2720,8062,No
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,98136,47.5208,-122.393,1360,5000,No
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,98074,47.6168,-122.045,1800,7503,No


So now we have a column 'house_renovation' which gives a boolean value,yes or no instead of 'yr_renovated column'.



We will introduce a function that we will use to check for the null and duplicated values in the dataframe.

In [9]:
def check_null_and_duplicates(df):
    """
    Check for both null values and duplicated rows in a DataFrame.
    
    Parameters:
    - df: DataFrame
        The DataFrame to check.
        
    Returns:
    - info_df: DataFrame
        DataFrame containing information about null values and duplicated rows.
    """
    # Check for null values
    null_values = df.isnull().sum()
    if null_values.sum() > 0:
        print("Null Values:")
        print(null_values)
    else:
        print("No Null Values Found.")

    # Check for duplicated rows
    duplicated_rows = df.duplicated().sum()
    if duplicated_rows > 0:
        print("\nDuplicated Rows Found:", duplicated_rows)
        duplicated_df = df[df.duplicated()]
        print(duplicated_df)
    else:
        print("\nNo Duplicated Rows Found.")
        duplicated_df = pd.DataFrame()  # Empty DataFrame if no duplicates
    
    # Create a DataFrame to store information
    info_df = pd.DataFrame({
        'Column': null_values.index,
        'Null Count': null_values.values,
        'Duplicated Rows': duplicated_rows
    })
    return info_df
info_df= check_null_and_duplicates(df) 
print (info_df)

Null Values:
id                     0
date                   0
price                  0
bedrooms               0
bathrooms              0
sqft_living            0
sqft_lot               0
floors                 0
waterfront          2376
view                  63
condition              0
grade                  0
sqft_above             0
sqft_basement          0
yr_built               0
zipcode                0
lat                    0
long                   0
sqft_living15          0
sqft_lot15             0
house_renovation       0
dtype: int64

No Duplicated Rows Found.
              Column  Null Count  Duplicated Rows
0                 id           0                0
1               date           0                0
2              price           0                0
3           bedrooms           0                0
4          bathrooms           0                0
5        sqft_living           0                0
6           sqft_lot           0                0
7             floors  

.No duplicated values were found.


## Function for dropping duplicates,nulls and column names.
So we will use the Python function 'dropper'. This function is used for cleaning a dataframe by dropping duplicates,null values and separated columns.

In [10]:
def dropper(df, one=None, two=None, three=None):
    '''
    Input: DataFrame, request 1,request 2, request 3
    requests:
    'duplicates' to drop duplicates
    'nulls' to drop null values
    list containing df column names l = ['','','']
    '''
    request = [one,two,three]
    if 'duplicates' in request:
        df = df.drop_duplicates()
    if 'nulls' in request:
        df = df.dropna()
    for req in request:
        if isinstance(req, list):
            df = df.drop(columns=req, axis=1).reset_index(drop=True)
    return(df)
data_info = check_dtypes(df)
print(df)

               id        date     price  bedrooms  bathrooms  sqft_living  \
0      7129300520  10/13/2014  221900.0         3       1.00         1180   
1      6414100192   12/9/2014  538000.0         3       2.25         2570   
2      5631500400   2/25/2015  180000.0         2       1.00          770   
3      2487200875   12/9/2014  604000.0         4       3.00         1960   
4      1954400510   2/18/2015  510000.0         3       2.00         1680   
...           ...         ...       ...       ...        ...          ...   
21592   263000018   5/21/2014  360000.0         3       2.50         1530   
21593  6600060120   2/23/2015  400000.0         4       2.50         2310   
21594  1523300141   6/23/2014  402101.0         2       0.75         1020   
21595   291310100   1/16/2015  400000.0         3       2.50         1600   
21596  1523300157  10/15/2014  325000.0         2       0.75         1020   

       sqft_lot  floors waterfront  view  ...          grade sqft_above  \


We went ahead and dropped our null values and since we did not have any duplicates they were not dropped. 